In [1]:
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 24.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=ceb063448536527799ac7889049fb9312628784470867aaf7ca39b3c86cd60a5
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [4]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

tar: spark-3.3.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [10]:
! mkdir ~/.kaggle

In [11]:
! cp /content/gdrive/MyDrive/kaggle_api/kaggle.json ~/.kaggle/kaggle.json

In [12]:
! chmod 600 ~/.kaggle/kaggle.json

In [14]:
! kaggle datasets download -d imakash3011/customer-personality-analysis

  0% 0.00/62.0k [00:00<?, ?B/s]
100% 62.0k/62.0k [00:00<00:00, 782kB/s]


In [15]:
! unzip customer-personality-analysis.zip

Archive:  customer-personality-analysis.zip
  inflating: marketing_campaign.csv  


In [138]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NomeDoSeuApp") \
    .getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [139]:
path = '../content/marketing_campaign.csv'
df_customer = spark.read.format('csv').option('sep', '\t').option('header', 'true').load(path)

In [140]:
df_customer.show()

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

In [141]:
df_customer = df_customer.drop('MntWines','MntFruits','MntMeatProducts','MntFishProducts')

In [142]:
df_customer = df_customer.where(col('Income').isNotNull())

In [161]:
#Quantidade de linhas do Dataframe
df_customer.count()

2216

In [144]:
#Para o primeiro indicador, precisamos transformar nossas colunas de Data e, DataType
df_customer = df_customer.withColumn('Year_Birth', to_date('Year_Birth'))

#Para o primeiro indicador, precisamos transformar nossas colunas de Data de consumo wm DateType
df_customer = df_customer.withColumn('Dt_Customer', col('Dt_Customer').cast(StringType())).withColumn('Dt_Customer', to_date('Dt_Customer', 'dd-MM-yyyy')).withColumn('Dy_Customer', year('Dt_Customer'))

#Calculamos a idade estimada do cliente
df_customer = df_customer.withColumn('AgeStimated', round(datediff('Dt_Customer','Year_Birth')/365))

In [162]:
df_customer.write.csv('/content/gdrive/MyDrive/marketingTratment.csv')